In [ ]:

# mean/box rating per month / yr agg by gender

# korelace poctu her a ratingu

In [1]:
import glob
import pandas as pd

data_files = glob.glob("chess_ratings/*.csv")
all_dfs = [pd.read_csv(file, sep='|') for file in data_files]

/tmp/ipykernel_83297/750563276.py:5: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  all_dfs = [pd.read_csv(file, sep='|') for file in data_files]


In [2]:
from spark_utils import preprocess_data

data = pd.concat(all_dfs)
data = preprocess_data(data)

In [3]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master('local[*]') \
    .config("spark.driver.memory", "20g") \
    .appName('chess-app') \
    .getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/05/20 13:04:08 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/05/20 13:04:09 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [4]:
spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", "true")

df = spark.createDataFrame(data)

In [6]:
from spark_utils import get_dataset

feature_cols = [('Flag', 'str'), ('Gms', 'sum'), ('Gms', 'mean'), ('Rat', 'mean'), ('tit', 'max'), ('wtit', 'max')]

agg_df = get_dataset(df, feature_cols)

22/05/20 13:05:02 WARN TaskSetManager: Stage 0 contains a task of very large size (32091 KiB). The maximum recommended task size is 1000 KiB.
22/05/20 13:05:23 WARN TaskSetManager: Stage 11 contains a task of very large size (32091 KiB). The maximum recommended task size is 1000 KiB.


Counts before join: base_df 345770, reduced 345770


22/05/20 13:05:31 WARN TaskSetManager: Stage 26 contains a task of very large size (32091 KiB). The maximum recommended task size is 1000 KiB.
22/05/20 13:05:36 WARN TaskSetManager: Stage 38 contains a task of very large size (32091 KiB). The maximum recommended task size is 1000 KiB.


Counts after join: base_df 345770


22/05/20 13:05:51 WARN TaskSetManager: Stage 49 contains a task of very large size (32091 KiB). The maximum recommended task size is 1000 KiB.


...

22/05/20 13:07:21 WARN TaskSetManager: Stage 274 contains a task of very large size (32091 KiB). The maximum recommended task size is 1000 KiB.
22/05/20 13:07:30 WARN TaskSetManager: Stage 310 contains a task of very large size (32091 KiB). The maximum recommended task size is 1000 KiB.


Counts after join: base_df 345770


22/05/20 13:07:43 WARN TaskSetManager: Stage 321 contains a task of very large size (32091 KiB). The maximum recommended task size is 1000 KiB.
22/05/20 13:07:54 WARN TaskSetManager: Stage 366 contains a task of very large size (32091 KiB). The maximum recommended task size is 1000 KiB.


Counts before join: base_df 345770, reduced 345770


Counts after join: base_df 345770


In [7]:
agg_df = agg_df.toPandas()

22/05/20 13:08:04 WARN TaskSetManager: Stage 408 contains a task of very large size (32091 KiB). The maximum recommended task size is 1000 KiB.


In [9]:
# filling newly created nans
for c in agg_df.columns:
    if 'Flag' in c:
        agg_df[c] = agg_df[c].fillna('i')
    else:
        agg_df[c] = agg_df[c].fillna(0)